In [1]:
import json
import random

from dotenv import load_dotenv
from google import genai
from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search
from google.genai import types
from pydantic import BaseModel, Field

In [2]:
load_dotenv()

retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

In [3]:
class FootballClubList(BaseModel):
    clubs: list[str] = Field(
        description="A list of exactly 10 major European football club names."
    )

In [4]:
client = genai.Client()

agent_sportscaster = Agent(
    name="sportscaster",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config,
    ),
    instruction="You are a sportscaster.",
)
runner_sportscaster = InMemoryRunner(agent=agent_sportscaster)

agent_lister = Agent(
    name="lister",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config,
    ),
    instruction="List 10 major European football clubs.",
    output_schema=FootballClubList,
)
runner_lister = InMemoryRunner(agent=agent_lister)


In [5]:
response_lister_raw = await runner_lister.run_debug(
    "List 10 major European football clubs"
)
response_lister = FootballClubList.model_validate_json(response_lister_raw[0].content.parts[0].text)
clubs =response_lister.clubs
print("clubs:", clubs)


 ### Created new session: debug_session_id

User > List 10 major European football clubs
lister > {
  "clubs": [
    "Real Madrid",
    "FC Barcelona",
    "Manchester United",
    "Liverpool FC",
    "Bayern Munich",
    "Juventus FC",
    "AC Milan",
    "Inter Milan",
    "Paris Saint-Germain",
    "Borussia Dortmund"
  ]
}
clubs: ['Real Madrid', 'FC Barcelona', 'Manchester United', 'Liverpool FC', 'Bayern Munich', 'Juventus FC', 'AC Milan', 'Inter Milan', 'Paris Saint-Germain', 'Borussia Dortmund']


In [6]:
print("Select a club:")
for i, club in enumerate(clubs, 1):
    print(f"{i}. {club}")

selection = int(input("Enter number: "))
human_club = clubs[selection - 1]
print(f"You selected: {human_club}")

Select a club:
1. Real Madrid
2. FC Barcelona
3. Manchester United
4. Liverpool FC
5. Bayern Munich
6. Juventus FC
7. AC Milan
8. Inter Milan
9. Paris Saint-Germain
10. Borussia Dortmund
You selected: Manchester United


In [7]:
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=f"What's the arch rival of {human_club}? Just give me the name of the club.",
)
computer_club = response.text
print(f"The arch rival of {human_club} is {computer_club}.")

The arch rival of Manchester United is Liverpool.


In [8]:
agent_computer = Agent(
    name="computer",
    model=Gemini(
        model="gemini-2.5-pro",
        retry_options=retry_config,
    ),
    instruction=f"""
You are the manager of the football club {computer_club}, and you are responsible for the draft. Your goal is to assemble the strongest team possible.

Draft Structure: You will draft exactly 18 players across 3 rounds, with each round offering a dynamically generated pool of 16 players. You can stop early in a round if the remaining players are not good enough.

Player Attributes: Each player has one or more natural positions (Forward, Midfielder, Defender, Goalkeeper) and a base star rating (1–6).

Opponent: H (human) is your opponent. Draft picks alternate, and you cannot select a player who has already been chosen.

Team Requirement: Your final roster must include at least 1 Goalkeeper (GK) in the starting lineup.

Star Rating Calculations (Effective Rating)

The player's effective star rating changes based on the position they are assigned:

Natural Position: Full base star rating.

Out-of-Position (Non-GK): Base stars minus 1 (e.g., a Midfielder playing Defender).

Non-Goalkeeper (Non-GK) playing as GK: Automatically reduced to 1 star.

Goalkeeper (GK) playing as a Non-GK: Automatically reduced to 1 star.
    """,
)
runner_computer = InMemoryRunner(agent=agent_computer)

agent_administrator = Agent(
    name="administrator",
    model=Gemini(
        # model="gemini-2.5-flash-lite",
        model="gemini-2.5-pro",
        retry_options=retry_config,
    ),
    instruction=f"""
        You are the administrator of the draft and the match between {human_club} and {computer_club}.
        You have the capability to create pools of real players with varying skill levels.
        The star ratings of the players should be distributed in a pyramid structure across the pool, with top-rated players at the top.
        These players can be from any team in the world. They can be from the past or present.
        Once a player is shown, he must not be shown again.
        Player positions and stars:
            * Natural positions: Forward, Midfielder, Defender, Goalkeeper
            * Stars: 1–6
            * A play can have one or multiple natural positions
    """,
)
runner_administrator = InMemoryRunner(agent=agent_administrator)

In [9]:
response_sportscaster_raw = await runner_sportscaster.run_debug(
    f"There is a draft and a match between {human_club} and {computer_club}. Briefly introduce the history of both clubs."
)


 ### Created new session: debug_session_id

User > There is a draft and a match between Manchester United and Liverpool. Briefly introduce the history of both clubs.
sportscaster > Alright, let's get into the heart of the action! We're talking about a draft and a massive match between two titans of English football: Manchester United and Liverpool! This isn't just any game; this is a fixture steeped in history, rivalry, and pure passion.

First, let's talk about **Manchester United**. These Red Devils, based in the vibrant city of Manchester, are one of the most successful clubs in the world. Founded way back in 1878 as Newton Heath LYR Football Club, they eventually rebranded as Manchester United in 1902. They've graced the top flight for an astonishingly long time, amassing an incredible trophy cabinet. We're talking multiple league titles, numerous FA Cups, and, of course, those iconic Champions League triumphs. Think legends like Sir Bobby Charlton, George Best, Denis Law, and the

In [10]:
human_squad = []
computer_squad = []

In [11]:
def human_choose_player(players):
    while True:
        player_choice = input("Choose a player (or stop to finish this round): ").strip()
        if player_choice in players:
            player_info = players[player_choice]
            print(f"You selected: {player_choice}")
            print(f"Player info: {player_info}")
            players.pop(player_choice)
            return {player_choice: player_info}
        elif player_choice == "stop":
            return None
        else:
            print("Invalid player. Please try again.")

async def computer_choose_player(runner_computer, players):
    available_players_str = json.dumps(players, indent=2)
    response = await runner_computer.run_debug(f"Available players: {available_players_str}\nChoose one player name from the list. Return only the name or the word stop to finishe this round if the remaining players are not good enough.")
    player_choice = response[0].content.parts[0].text.strip()
    if player_choice in players:
        print(f"Computer selected: {player_choice}")
        player_info = players[player_choice]
        print(f"Player info: {player_info}")
        players.pop(player_choice)
        return {player_choice: player_info}
    return None


In [12]:
def print_team_formation(players):
    """
    Prints players grouped by position in the order: 
    Forward, Midfielder, Defender, Goalkeeper.
    
    Args:
        players: List of dicts or objects with 'name', 'position', and 'stars'.
    """
    # 1. Define the explicit print order
    order = ["Forward", "Midfielder", "Defender", "Goalkeeper"]
    
    # 2. Group players by position
    # Use a dictionary to hold lists for each position
    squad = {pos: [] for pos in order}

    for player in players:
        for name, info in player.items():
            positions = info['positions']
            for pos in positions:
                if pos in squad:
                    squad[pos].append([name, info])
                    break
    
    # 3. Print strictly in the defined order
    for pos in order:
        if squad[pos]:  # Only print if there are players in this position
            print(f"{pos}s")
            print("-" * 20)
            for each in squad[pos]:
                print(each)
            print()

In [13]:
async def draft_players():
    response_administrator_raw = await runner_administrator.run_debug(
        f"""
        Create a pool of 16 real players with varying skill levels from 1 star to 6 stars.
        These players will be used for the draft.
        Although these players may play on a team in reality, for this draft they are all free agents and available to be selected.
        Each player's name, natural position(s), and star rating should be shown.
        Use a pyramid distribution where higher-rated players are increasingly rare:
        6★ players should be extremely rare (elite/world class talent)
        5★ players should be rare (excellent)
        4★ players should be uncommon (very good)
        3★ players should be fairly common (average/solid)
        2★ and 1★ players should make up the bulk of the pool (below average to poor)

        The output must be a dictionary, whose keys are the player names and values are dictionaries with the keys "positions" and "stars".
        The output mustn't include "```json" or "```". 
        """
    )
    response_administrator = response_administrator_raw[0].content.parts[0].text.replace("```json", "").replace("```", "")
    players = json.loads(response_administrator)
    for name, info in players.items():
        print(name, info)

    human_finished = False
    computer_finished = False

    while players and (not human_finished or not computer_finished):
        if len(human_squad) >= 18:
            human_finished = True
        if len(computer_squad) >= 18:
            computer_finished = True

        print()
        print("players to choose from:")
        for name, info in players.items():
            print(name, info)
        print()

        if not human_finished:
            human_choice = human_choose_player(players)
            if human_choice:
                human_squad.append(human_choice)
            else:
                human_finished = True

        if not computer_finished:
            computer_choice = await computer_choose_player(runner_computer, players)
            if computer_choice:
                computer_squad.append(computer_choice)
            else:
                computer_finished = True

        print("human_squad size:", len(human_squad))
        print("computer_squad size:", len(computer_squad))
        print()

        print("human_squad:")
        print_team_formation(human_squad)
        print("computer_squad:")
        print_team_formation(computer_squad)
        print()


In [14]:
for i in range(3):
    await draft_players()


 ### Created new session: debug_session_id

User > 
        Create a pool of 16 real players with varying skill levels from 1 star to 6 stars.
        These players will be used for the draft.
        Although these players may play on a team in reality, for this draft they are all free agents and available to be selected.
        Each player's name, natural position(s), and star rating should be shown.
        Use a pyramid distribution where higher-rated players are increasingly rare:
        6★ players should be extremely rare (elite/world class talent)
        5★ players should be rare (excellent)
        4★ players should be uncommon (very good)
        3★ players should be fairly common (average/solid)
        2★ and 1★ players should make up the bulk of the pool (below average to poor)

        The output must be a dictionary, whose keys are the player names and values are dictionaries with the keys "positions" and "stars".
        The output mustn't include "```json" or "```".

In [15]:
human_squad

[{'Pelé': {'positions': ['Forward'], 'stars': 6}},
 {'Paolo Maldini': {'positions': ['Defender'], 'stars': 5}},
 {'Kevin De Bruyne': {'positions': ['Midfielder'], 'stars': 4}},
 {'Jordan Pickford': {'positions': ['Goalkeeper'], 'stars': 3}},
 {'James Ward-Prowse': {'positions': ['Midfielder'], 'stars': 3}},
 {'Lionel Messi': {'positions': ['Forward'], 'stars': 6}},
 {'Franz Beckenbauer': {'positions': ['Defender'], 'stars': 5}},
 {'Luka Modrić': {'positions': ['Midfielder'], 'stars': 4}},
 {'Aaron Wan-Bissaka': {'positions': ['Defender'], 'stars': 3}},
 {'Nick Pope': {'positions': ['Goalkeeper'], 'stars': 3}},
 {'Cristiano Ronaldo': {'positions': ['Forward'], 'stars': 6}},
 {'Manuel Neuer': {'positions': ['Goalkeeper'], 'stars': 5}},
 {'Casemiro': {'positions': ['Midfielder'], 'stars': 4}},
 {'Robert Lewandowski': {'positions': ['Forward'], 'stars': 4}},
 {'Kurt Zouma': {'positions': ['Defender'], 'stars': 3}},
 {'Jordan Henderson': {'positions': ['Midfielder'], 'stars': 3}},
 {'Richar

In [16]:
computer_squad

[{'Zinedine Zidane': {'positions': ['Midfielder'], 'stars': 5}},
 {'Alisson Becker': {'positions': ['Goalkeeper'], 'stars': 4}},
 {'Virgil van Dijk': {'positions': ['Defender'], 'stars': 4}},
 {'Olivier Giroud': {'positions': ['Forward'], 'stars': 3}},
 {'Conor Coady': {'positions': ['Defender'], 'stars': 3}},
 {'Scott McTominay': {'positions': ['Midfielder'], 'stars': 2}},
 {'Wout Weghorst': {'positions': ['Forward'], 'stars': 2}},
 {'Seamus Coleman': {'positions': ['Defender'], 'stars': 2}},
 {'Ronaldinho': {'positions': ['Forward', 'Midfielder'], 'stars': 5}},
 {'Harry Kane': {'positions': ['Forward'], 'stars': 4}},
 {"N'Golo Kanté": {'positions': ['Midfielder'], 'stars': 4}},
 {'Diogo Jota': {'positions': ['Forward'], 'stars': 3}},
 {'James Maddison': {'positions': ['Midfielder'], 'stars': 3}},
 {'Harry Maguire': {'positions': ['Defender'], 'stars': 2}},
 {'Divock Origi': {'positions': ['Forward'], 'stars': 2}},
 {'Ross Barkley': {'positions': ['Midfielder'], 'stars': 2}},
 {'Johan

In [17]:
def get_squad_stars(squad):
    stars = 0
    # player is a dict: the key is the player name, the value is a dict with keys "positions" and "stars"
    for player in squad:
        for name, info in player.items():
            stars += info['stars']
    return stars

human_squad_stars = get_squad_stars(human_squad)
computer_squad_stars = get_squad_stars(computer_squad)

print("human_squad_stars:", human_squad_stars)
print("computer_squad_stars:", computer_squad_stars)

human_squad_stars: 72
computer_squad_stars: 59


In [18]:
print_team_formation(human_squad)

Forwards
--------------------
['Pelé', {'positions': ['Forward'], 'stars': 6}]
['Lionel Messi', {'positions': ['Forward'], 'stars': 6}]
['Cristiano Ronaldo', {'positions': ['Forward'], 'stars': 6}]
['Robert Lewandowski', {'positions': ['Forward'], 'stars': 4}]
['Richarlison', {'positions': ['Forward'], 'stars': 3}]

Midfielders
--------------------
['Kevin De Bruyne', {'positions': ['Midfielder'], 'stars': 4}]
['James Ward-Prowse', {'positions': ['Midfielder'], 'stars': 3}]
['Luka Modrić', {'positions': ['Midfielder'], 'stars': 4}]
['Casemiro', {'positions': ['Midfielder'], 'stars': 4}]
['Jordan Henderson', {'positions': ['Midfielder'], 'stars': 3}]

Defenders
--------------------
['Paolo Maldini', {'positions': ['Defender'], 'stars': 5}]
['Franz Beckenbauer', {'positions': ['Defender'], 'stars': 5}]
['Aaron Wan-Bissaka', {'positions': ['Defender'], 'stars': 3}]
['Kurt Zouma', {'positions': ['Defender'], 'stars': 3}]
['Jonny Evans', {'positions': ['Defender'], 'stars': 2}]

Goalkeepers

In [19]:
print_team_formation(computer_squad)

Forwards
--------------------
['Olivier Giroud', {'positions': ['Forward'], 'stars': 3}]
['Wout Weghorst', {'positions': ['Forward'], 'stars': 2}]
['Ronaldinho', {'positions': ['Forward', 'Midfielder'], 'stars': 5}]
['Harry Kane', {'positions': ['Forward'], 'stars': 4}]
['Diogo Jota', {'positions': ['Forward'], 'stars': 3}]
['Divock Origi', {'positions': ['Forward'], 'stars': 2}]
['Johan Cruyff', {'positions': ['Forward', 'Midfielder'], 'stars': 5}]

Midfielders
--------------------
['Zinedine Zidane', {'positions': ['Midfielder'], 'stars': 5}]
['Scott McTominay', {'positions': ['Midfielder'], 'stars': 2}]
["N'Golo Kanté", {'positions': ['Midfielder'], 'stars': 4}]
['James Maddison', {'positions': ['Midfielder'], 'stars': 3}]
['Ross Barkley', {'positions': ['Midfielder'], 'stars': 2}]

Defenders
--------------------
['Virgil van Dijk', {'positions': ['Defender'], 'stars': 4}]
['Conor Coady', {'positions': ['Defender'], 'stars': 3}]
['Seamus Coleman', {'positions': ['Defender'], 'stars'